In [22]:
import numpy as np
import pandas as pd

In [23]:
credits = pd.read_csv('credits.csv')

ParserError: Error tokenizing data. C error: out of memory

In [24]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [ ]:
meta = pd.read_csv('movie_metadata.csv')

In [ ]:
meta['release_date'] = pd.to_datetime(meta['title_year'], errors='coerce')

In [25]:

meta['year'] = meta['release_date'].dt.year

In [26]:
meta['year'].value_counts().sort_index()

1970.0    4935
Name: year, dtype: int64

In [27]:
# Getting only movies upto 2017 movies as we don't have enough data for the movies from 2018, 2019 and 2020. 
# We'll deal with it in the upcoming preprocessing files
new_meta = meta.loc[meta.year <= 2017,['genres','id','title','year']]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['id', 'title'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [ ]:

new_meta

In [ ]:
new_meta['id'] = new_meta['id'].astype(int)

In [ ]:
data = pd.merge(new_meta, credits, on='id')

In [ ]:
pd.set_option('display.max_colwidth', 75)
data

In [ ]:
# evaluates an expression node or a string containing a Python literal or container display
import ast
data['genres'] = data['genres'].map(lambda x: ast.literal_eval(x))
data['cast'] = data['cast'].map(lambda x: ast.literal_eval(x))
data['crew'] = data['crew'].map(lambda x: ast.literal_eval(x))

In [ ]:
def make_genresList(x):
    gen = []
    st = " "
    for i in x:
        if i.get('name') == 'Science Fiction':
            scifi = 'Sci-Fi'
            gen.append(scifi)
        else:
            gen.append(i.get('name'))
    if gen == []:
        return np.NaN
    else:
        return (st.join(gen))

In [ ]:
data['genres_list'] = data['genres'].map(lambda x: make_genresList(x))

In [ ]:
data['genres_list']

In [ ]:
def get_actor1(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == []:
        return np.NaN
    else:
        return (casts[0])

In [ ]:
data['actor_1_name'] = data['cast'].map(lambda x: get_actor1(x))

In [ ]:
def get_actor2(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=1:
        return np.NaN
    else:
        return (casts[1])

In [ ]:
data['actor_2_name'] = data['cast'].map(lambda x: get_actor2(x))

In [ ]:
data['actor_2_name']

In [ ]:
def get_actor3(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=2:
        return np.NaN
    else:
        return (casts[2])

In [ ]:
data['actor_3_name'] = data['cast'].map(lambda x: get_actor3(x))

In [ ]:
data['actor_3_name']

In [ ]:
def get_directors(x):
    dt = []
    st = " "
    for i in x:
        if i.get('job') == 'Director':
            dt.append(i.get('name'))
    if dt == []:
        return np.NaN
    else:
        return (st.join(dt))

In [ ]:
data['director_name'] = data['crew'].map(lambda x: get_directors(x))

In [ ]:
data['director_name']

In [ ]:
movie = data.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres_list','title']]

In [ ]:
movie

In [ ]:
movie.isna().sum()

In [ ]:
movie = movie.dropna(how='any')

In [ ]:
movie.isna().sum()

In [ ]:
movie = movie.rename(columns={'genres_list':'genres'})
movie = movie.rename(columns={'title':'movie_title'})

In [ ]:
movie['movie_title'] = movie['movie_title'].str.lower()

In [ ]:
movie['comb'] = movie['actor_1_name'] + ' ' + movie['actor_2_name'] + ' '+ movie['actor_3_name'] + ' '+ movie['director_name'] +' ' + movie['genres']

In [ ]:
movie

In [ ]:
movie.drop_duplicates(subset ="movie_title", keep = 'last', inplace = True)

In [ ]:
movie

In [ ]:
movie.to_csv('movie.csv',index=False)